In [4]:
''' 
 Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering
 Date; 9. 24. 2021 - 11. 13. 2021
 Title: Artificial Intelligence_Project 2
 Professor: Seung-Hoon Na
 Reference: https://cpm0722.github.io/pytorch-implementation/transformer
            https://pytorch.org/tutorials/beginner/transformer_tutorial.html'''

' \n Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering\n Date; 9. 24. 2021 - 11. 13. 2021\n Title: Artificial Intelligence_Project 2\n Professor: Seung-Hoon Na\n Reference: https://cpm0722.github.io/pytorch-implementation/transformer\n            https://pytorch.org/tutorials/beginner/transformer_tutorial.html'

In [5]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# Encoder
 - Generate context (vector made by Encoding)
 - compress all of the information in sentences without leakage
 - pad mask: do not assign attention to pad token

# Decoder
- make a sentence from sentence, context
- INPUT: context(output of Encoder), some sentence(Ground Truth[:-1]'s sentence)
- subsequent mask: for hiding token's solution (future word)

# Teacher Forcing: use labeled data(Ground Truth) as input in Supervised Learning

# Self-Attention
- 'Self': find attention in a same sentence
- Attention: guess what a word(ex. it or the thrid person singular) points in a sentecne
- Query: now token
- Key: target token to find attention
- Value: same Key

In [18]:
class TransformerModel(nn.Module):
    
    def __init__(self, ntoken, d_model, nhead, d_hid, nlayers, dropout):
                     # (int)    (int)   (int)  (int)   (int)    (float)     
        super().__init__()
        # info
        self.model_type = 'Transformer'
        
        # layers
        self.pos_encoder = PositionalEncoding(d_model, dropout) # include information of word's location
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)
        
        self.init_weights()
    
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
        
    def forward(self, src, src_mask):
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [1]:
def generate_square_subsequent_mask(sz: int):#->Tensor
    """
    Generates an upper-triangular matrix of -inf, with zeros on diag.
    triu: Generates an upper-triangular matrix
    """
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal = 1)

'''
ex sz == 5
tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])
'''

'\nex sz == 5\n'

# Transformer’s Input (Positional Encoding)
- purpose of positional encoding: normalize positional information -> limit specific range(even index: sin, odd indxe: cos)

# Positional Encoding
- Assign importance to element depending on location

In [8]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p = dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):#-> Tensor
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Load and batch data

In [9]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Conclusion: vocab = word embedding

In [10]:
train_iter = WikiText2(split='train') # load data
tokenizer = get_tokenizer('basic_english') # make tokens
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>']) 
'''
 map(fucntion, iterable(list, tuple etc))
 conduct function until iterable 
'''
vocab.set_default_index(vocab['<unk>'])

# Make data

In [11]:
def data_process(raw_text_iter): #: dataset.IterableDataset) -> Tensor
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))
'''
torch.numel: Returns the total number of elements in the input tensor.
filter(function, iterable): 
torch.cat(tensor, dim, out):  
'''
# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
'''
train_iter = WikiText2(split='train')
val_iter = WikiText2(split='val')
test_iter = WikiText2(split='test')
'''
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Batch data

In [13]:
def batchify(data, bsz: int): # -> Tensor
    """Divides the data into bsz separate sequences, removing extra elements that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(-1, seq_len).t().contiguous()
    '''
    tensor.view: Returns a new tensor with the same data as the self tensor but of a different shape
    torch.t:
     - Expects input to be <= 2-D tensor and transposes dimensions 0 and 1.
     - 0-D and 1-D tensors are returned as is. 
       When input is a 2-D tensor this is equivalent to transpose(input, 0, 1).
    contiguous: Returns a contiguous in memory tensor containing the same data as self tensor
    '''
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

# Functions to generate input and target sequence

# BPTT
 - 
 - 
 - 
 - 
 - 

In [14]:
bptt = 35 # chunks of length
def get_batch(source, i): # -> Tuple[Tensor, Tensor]
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

# Initiate an instance

In [19]:
ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # select some of nueral network units stochastically

model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

# Run the model
- CrossEntropyLoss with the SGD (stochastic gradient descent) optimizer. 
- The learning rate: initially to 5.0 and follows a StepLR schedule. 
- During training, we use 'nn.utils.clip_grad_norm_ ' to prevent gradients from exploding.

In [25]:
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma = 0.95)
'''
lr_scheduler.StepLR:
 Decays the learning rate of each parameter group by gamma every step_size epochs.
'''

def train(model):
    model.train()  # turn on train mode
    
    '''
    '''
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    num_batches = len(train_data) // bptt
    
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask) # get hypothesis
        loss = criterion(output.view(-1, ntokens), targets) # reshape 

        optimizer.zero_grad() # for initialize gradient to newly calculate gradient whenever repeat
        loss.backward() # calculate loss with back propagation
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        # 
        optimizer.step() # 

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model, eval_data): # -> float
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [26]:
best_val_loss = float('inf')
epochs = 3
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 474.38 | loss  8.18 | ppl  3578.34
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 505.03 | loss  6.92 | ppl  1011.33


KeyboardInterrupt: 